In [66]:
#https://towardsdatascience.com/bert-for-measuring-text-similarity-eec91c6bf9e1#:~:text=The%20logic%20is%20this%3A,them%20%E2%80%94%20more%20on%20that%20here.
#https://www.youtube.com/watch?v=ZxR38An5TQE

In [2]:
import os
os.chdir("/content/drive/MyDrive/hackathon/nlp_hack")

In [3]:
file = open("TrainData.txt","r",encoding='utf-8')
data=file.read()

In [4]:
content_list = data.splitlines()

Store the Question ans answer in different list

In [5]:
qstn=[]
anwr=[]
for lst in content_list[1:]:
  q=lst.split('\t')[-1]
  a=lst.split('\t')[0]
  qstn.append(q)
  anwr.append(a)

In [6]:
qstn[:2]

["The weather is great isn't it?", "that one. the one that's all black."]

In [7]:
import pandas as pd
import re

def clean_sentence(sentence):
  sentence=sentence.lower().strip()
  sentence=re.sub(r'[^a-z0-9\s]','',sentence)
  return sentence

In [25]:
def get_clean_sentences(df):
  sents=df["question"]
  clean_sentences=[]
  for index,row in df.iterrows():
    cleaned=clean_sentence(row["question"])
    clean_sentences.append(cleaned)
  return clean_sentences

# creating a Dataframe

In [9]:
df=pd.DataFrame()
df["question"]=qstn
df["answer"]=anwr
df.head()

,question,answer
0,The weather is great isn't it?,Yes. It's absolutely beautiful today.
1,that one. the one that's all black.,"yes, i like that one, too."
2,i got it from macy's.,it's really nice.
3,when does it start?,at 8:00 p.m.
4,what's the matter with green eyes?,"nothing, except my favorite color is blue."


Pretrain bert  model

In [18]:
pip install sentence-transformers

     |████████████████████████████████| 81kB 4.9MB/s 
     |████████████████████████████████| 2.3MB 17.1MB/s 
     |████████████████████████████████| 1.2MB 37.3MB/s 
     |████████████████████████████████| 901kB 46.5MB/s 
     |████████████████████████████████| 3.3MB 43.2MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.1.1-cp37-none-any.whl size=123338 sha256=c79e656368900dddb3a8d5615d79049e82e57f17bc45031c8330ff1ff2125b52
  Stored in directory: /root/.cache/pip/wheels/5e/89/29/45e45adc162b50f97f71801e8b07947c9cfe2b3ae7dbf37896
Successfully built sentence-transformers


Use bert-base-nli-mean-tokens model

In [19]:

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/sbert.net_models_bert-base-nli-mean-tokens/0_BERT were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Example of encoding using pretrain bert model

In [21]:
sentence_embeddings = model.encode(["how are you","Are you ok"])
sentence_embeddings

array([[ 0.27128318, -0.62632734,  2.23879   , ..., -0.34569022,
        -0.11973493, -0.07137005],
       [ 0.47712335, -0.90244025,  2.24266   , ...,  0.12103744,
        -0.46206427,  0.02697023]], dtype=float32)

#embedded training Question

In [26]:
from tqdm import tqdm
clean_sentences=get_clean_sentences(df)
bert_embedded=[]
for sent in tqdm(clean_sentences):
  bert_embedded.append(model.encode([sent]))

100%|██████████| 5495/5495 [01:33<00:00, 58.99it/s]


In [57]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
#this function compare the cosine similarity between the embedded question and the embedded training question 
def ans(ques_emb,sen_emb,faq,sen):
  max_sim=-1
  index_sim=-1
  for index,faq_emb in enumerate(sen_emb):
    sim=cosine_similarity(faq_emb,ques_emb)[0][0]
    #print(index,sim,sen[index])
    if sim>max_sim:
      max_sim=sim
      index_sim=index
  #print("\n")
  #print("Question:",question)
  #print("\n")
  #print("Retrieved:",faq.iloc[index_sim,0])
  #print(index_sim)
  #print(faq.iloc[index_sim,1])
  return faq.iloc[index_sim,1]

In [29]:
test = pd.read_csv('TestData.csv')
test.head()

,I.D.,Question
0,QN_1,"i'll give you a speech like that, too."
1,QN_2,"i know, you're absolutely right."
2,QN_3,i liked it.
3,QN_4,the baby was eight pounds six ounces.
4,QN_5,I was sold a wireless service unavailable in m...


In [31]:
test["Question"][1]

"i know, you're absolutely right."

In [58]:
no=542
question=clean_sentence(test["Question"][no])
print("test_question:",test["Question"][no])
qstn_emb=model.encode(question).reshape(1,-1)
ans(qstn_emb,bert_embedded,df,clean_sentences)

test_question: what do you mean?


'are there going to be layoffs at this place?'

In [59]:
test_qstn_list=test["Question"].to_list()

In [61]:
test_ansr_list=[]
for qstn in test_qstn_list:
  question=clean_sentence(qstn)
  qstn_emb=model.encode(question).reshape(1,-1)
  answer=ans(qstn_emb,bert_embedded,df,clean_sentences)
  test_ansr_list.append(answer)

In [62]:
test["Answer"]=test_ansr_list
test.head()

,I.D.,Question,Answer
0,QN_1,"i'll give you a speech like that, too.",do you think anyone will come to my funeral?
1,QN_2,"i know, you're absolutely right.",i wish it would cool off one day.
2,QN_3,i liked it.,"i'll give you a speech like that, too."
3,QN_4,the baby was eight pounds six ounces.,that's good to hear.
4,QN_5,I was sold a wireless service unavailable in m...,"I see it here,we charged you $5 extra a month."


In [63]:
test.tail()

,I.D.,Question,Answer
538,QN_539,at night i went out to eat. the food was delic...,people who live in hawaii are lucky.
539,QN_540,TELL ME ABOUT BASEBALL,What is Baseball
540,QN_541,"you'll remember to do it, but you won't have t...","okay, i'll take it out front right now."
541,QN_542,you are not smart,you may be right.
542,QN_543,what do you mean?,are there going to be layoffs at this place?


In [64]:
test=test.drop("Question",1)

In [65]:
test.to_csv('gunand_2.csv', header=True, index=False)